In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from dataclasses import dataclass
@dataclass
class KNN:
     k:int=3
     def fit(self,x,y):
      self.x=x
      self.y=y
     def predict(self,X_test):
      #print((self.x-X_test)**2)
      dis=np.sqrt((self.x-X_test)**2).sum(axis=1)
      print(dis)
      ind=np.argsort(dis)[:self.k]
      print(ind)
      print(np.bincount(self.y[ind]))
      print(np.argmax(np.bincount(self.y[ind])))
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([0, 1, 0, 1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
k=KNN()
k.fit(X_train,y_train)
k.predict(X_test)
print(X_train,y_train)
print(X_train.shape)

In [ ]:
df=pd.read_csv('/content/badminton_dataset.csv')
y=df.iloc[:,-1]
X=df.iloc[:,:-1]
print(X,y)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
@dataclass
class NaiveBayes:
  def fit(self,x,y):
    self.x=x.values
    self.y=y.values.ravel()
    self.data=pd.concat([x,y],axis=1).values
    #print(self.data)
    self.condprob=[]
    print(self.x.shape)
    for i in np.unique(y):
      #print(i)
      setattr(self, i ,len(self.y[self.y==i]))
      setattr(self, f"prob_{i}",len(self.y[self.y==i])/len(y))
      print(i,getattr(self,i))
      print(f"prob_{i}",getattr(self, f"prob_{i}"))
    for i in range(self.x.shape[1]):
          ip=self.x[:,i]
          #featureclassprobs={}
          for j in np.unique(ip):
            featureprobs={}
            for k in np.unique(y):
                fe=np.sum((self.data[:,i]==j)&(self.data[:,-1]==k))
                #featureprobs[j]=((fe)/(getattr(self,k)))
                setattr(self,f"{j}_{k}",((fe)/(getattr(self,k))))
                print(f"{j}_{k}",getattr(self,f"{j}_{k}"))
            #featureclassprobs[k]=featureprobs
          #self.condprob.append(featureclassprobs)
          print(ip)
          print(self.condprob)
  def predict(self,X_test):
      y_pred=np.array([])
      for X in X_test.values:
         print(X)
         totalres=[]
         result=1
         for j in np.unique(y):
            res=np.array([])
            for i in range(len(X)):
             #print(i,self.condprob[i][X[i]])
                res=np.append(res,getattr(self,f"{X[i]}_{j}"))#self.condprob[i][X[i]]
                result=res
            result=np.append(result,[getattr(self,f"prob_{j}")])
            totalres.append({j:np.prod(result)})
         print(totalres)
         #sum=0
         dic={}
         lis=[]
         for item in totalres:
            dic.update(item)
         #print(dic)

         total=sum([i for i in dic.values()])
         #print("hi",total)
         lis=[(label,(value/total)) for label,value in dic.items()]
         #print(lis)
         lis.sort(key=lambda x:x[1],reverse=True)
         print(lis[0][0])
         y_pred=np.append(y_pred,[lis[0][0]])
      return y_pred
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=5)
n=NaiveBayes()
n.fit(X,y)
print(n.condprob)
y_pred=n.predict(X_test)
print(accuracy_score(y_pred,y_test))
#print(X['astigmatic'].value_counts())


In [ ]:
class SGDRegressor:

    def __init__(self,learning_rate=0.01,epochs=100):

        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])

                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_

                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

import random

class MBGDRegressor:

    def __init__(self,batch_size,learning_rate=0.01,epochs=100):

        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epochs):

            for j in range(int(X_train.shape[0]/self.batch_size)):

                idx = random.sample(range(X_train.shape[0]),self.batch_size)

                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

class GDRegressor:

    def __init__(self,learning_rate=0.01,epochs=100):

        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            # update all the coef and the intercept
            y_hat = np.dot(X_train,self.coef_) + self.intercept_
            #print("Shape of y_hat",y_hat.shape)
            intercept_der = -2 * np.mean(y_train - y_hat)
            self.intercept_ = self.intercept_ - (self.lr * intercept_der)

            coef_der = -2 * np.dot((y_train - y_hat),X_train)/X_train.shape[0]
            self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [ ]:
from dataclasses import dataclass,field
@dataclass
class Node:
  feature:str=None
  threshold:str=None
  gain:int=None
  sample:int=None
  values:dict=None
  left:Node=None
  right:Node=None
  opclass:str=None
  isleaf:bool=False
@dataclass
class DecisionTreeClassifier:
            root=None
            def fit(self,X,y):
                data=pd.concat([X,y],ignore_index=False,axis=1)
                self.root=self.buildtree(data)

            def buildtree(self,data):
                if len(data.iloc[:,-1].unique())==1:
                        opclass=data.iloc[:,-1].unique()[0]
                        sample=data.iloc[:,-1].shape[0]
                        return Node(gain=0,values={opclass:sample},opclass=opclass,sample=sample,isleaf=True)


                print(data)

                leftdata,rightdata,bestgain,bestfeature,bestthreshold,resultvalue,opclass,sample=self.bestsplit(data)

                left=self.buildtree(leftdata)
                right=self.buildtree(rightdata)
                return Node(feature=bestfeature,threshold=bestthreshold,gain=bestgain,values=resultvalue,opclass=opclass,sample=sample,left=left,right=right)
            def bestsplit(self,data):
                print(data)
                op=data.iloc[:,-1].value_counts().to_dict()
                print(op)
                bestgain=-1
                global bestfeature
                global bestthreshold
                global resultvalue
                opclass=None
                sample=None
                for col in data.columns:

                  if col != data.columns[-1]:
                        threshold=data[col].unique()
                        for thres in threshold:
                            print(thres)
                            df=pd.crosstab(index=data[data[col]==thres][col],columns=data[data[col]==thres][data.columns[-1]],margins=True,margins_name='Sum')
                            df1=pd.crosstab(index=data[data[col]!=thres][col],columns=data[data[col]!=thres][data.columns[-1]],margins=True,margins_name='Sum')
                            dic={}
                            dic1={}
                            value={}
                            valuedash={}
                            coln=None
                            thresh=None
                            print('check df empty',df.empty)
                            print('check df1 empty',df1.empty)
                            for i in df.columns:
                                if i!='Sum':
                                    value[i]=df.loc['Sum',i]
                                    df[f"prob_{i}"]=df.loc['Sum',i]/df.loc['Sum','Sum']
                                    dic[f"prob_{thres}_{i}"]=df.loc['Sum',i]/df.loc['Sum','Sum']
                            if (not df1.empty):
                              for j in df1.columns:
                                if j!='Sum':

                                    valuedash[j]=df1.loc['Sum',j]
                                    df1[f"prob_{j}"]=df1.loc['Sum',j]/df1.loc['Sum','Sum']
                                    dic1[f"prob_not{thres}_{j}"]=df1.loc['Sum',j]/df1.loc['Sum','Sum']
                            if df1.empty:
                                gain,coln,thresh=self.informationgain(data,op,df,df1,dic,dic1,col,thres)
                            else:
                                gain,coln,thresh=self.informationgain(data,op,df,df1,dic,dic1,col,thres)
                            print("gain",gain)
                            print("bestgain",bestgain)

                            if gain>bestgain:
                                print(gain,bestgain)
                                bestgain=gain
                                bestfeature=coln
                                bestthreshold=thresh
                                print("bestfeature",bestfeature)
                                print("bestthreshold",bestthreshold)
                                if df1.empty:
                                   for key in value:
                                    resultvalue[key]=value[key]
                                    opclass=sorted(resultvalue.items(),key=lambda x:x[1],reverse=True )[0][0]
                                    sample=data.shape[0]
                                else:
                                    for o in op.keys():
                                            if o not in value.keys():
                                                value[o]=0
                                            if o not in valuedash.keys():
                                                valuedash[o]=0
                                    print(value,valuedash)
                                    resultvalue={}
                                    for key in value:
                                        resultvalue[key]=value[key]+valuedash[key]
                                    opclass=sorted(resultvalue.items(),key=lambda x:x[1],reverse=True )[0][0]
                                    sample=data.shape[0]





                leftdata=data[data[bestfeature]==bestthreshold]
                rightdata=data[data[bestfeature]!=bestthreshold]
                return(leftdata,rightdata,bestgain,bestfeature,bestthreshold,resultvalue,opclass,sample)
            def informationgain(self,data,op,df,df1,dic,dic1,col,thres):
                print(df)
                print()
                print(df1)
                print()
                weightedsump=df.loc['Sum','Sum']
                if df1.empty and len(dic1)==0:
                    totalsum=np.sum(list(op.values()))
                    dataentropy=self.entropy(list(np.array(list(op.values()))/totalsum))
                    e=self.entropy(list(dic.values()))
                    return ((dataentropy-(weightedsump/totalsum)*e),col,thres)
                weightedsumpdash=df1.loc['Sum','Sum']
                p=[]
                pdash=[]
                print(dic)


                totalsum=np.sum(list(op.values()))
                dataentropy=self.entropy(list(np.array(list(op.values()))/totalsum))
                e=self.entropy(list(dic.values()))
                edash=self.entropy(list(dic1.values()))
                print(dataentropy,e,edash)
                return ((dataentropy-(weightedsump/totalsum)*e-(weightedsumpdash/totalsum)*edash),col,thres)



            def entropy(self,prob):
                return np.sum([-(i)*(np.log2(i))for i in prob if i>0])

            def predict(self, X):
                   print(self._traverse_tree(X, self.root))
                   #return np.array([self._traverse_tree(x, self.root) for x in X])

            def _traverse_tree(self, x, node):
                if node.isleaf:
                    return node.opclass

                if x[node.feature] == node.threshold:
                    return self._traverse_tree(x, node.left)
                return self._traverse_tree(x, node.right)


import graphviz

def export_graphviz(root, filename='tree.png'):
    graph = graphviz.Digraph(format='png')
    recurse_tree(root, graph)
    graph.render(filename, cleanup=True)
    return filename

def recurse_tree(node, graph):
    if node.isleaf:
        graph.node(str(id(node)), label=f"{node.opclass}\nSamples = {node.sample}\nGain = {node.gain}", fillcolor="#ffffff")
    else:
        graph.node(str(id(node)), label=f"{node.feature}== {node.threshold}\nSamples = {node.sample}\nGain = {node.gain}", fillcolor="#ffffff")
        recurse_tree(node.left, graph)
        recurse_tree(node.right, graph)
        graph.edge(str(id(node)), str(id(node.left)), label="True")
        graph.edge(str(id(node)), str(id(node.right)), label="False")

dtc=DecisionTreeClassifier()
dtc.fit(X,y)
root=dtc.root
export_graphviz(root)

In [ ]:
class DecisionTree:
  def __init__(self,tree):
    self.root=None
    self.tree=tree
    self.queue=[]

  def valueCountsDict(self,df,column):
    valueCounts=np.unique(df[column],return_counts=True)
    valueCountsDict = dict(zip(valueCounts[0],valueCounts[1]))
    return valueCountsDict

  def entropy(self,df,column):
    entropy=0
    tempdf=pd.DataFrame(columns=self.valueCountsDict(df,df.iloc[:,-1].name).keys(),index=self.valueCountsDict(df,column).keys())
    for x in self.valueCountsDict(df,column).keys():
      for y in self.valueCountsDict(df,df.iloc[:,-1].name).keys():
        tempdf.loc[x,y]=len(df[(df[column]==x) & (df[df.iloc[:,-1].name]==y)].index)
    valueCounts=self.valueCountsDict(df,column)
    targetValueCounts=self.valueCountsDict(df,df.iloc[:,-1].name)
    total=len(df.index)
    for category in valueCounts.keys():
      prob=valueCounts[category]/total
      infoGain=0
      for targetCategory in targetValueCounts.keys():
        infoprob=tempdf.loc[category,targetCategory]/tempdf.loc[category].sum()
        if infoprob!=0:
          infoGain-=(infoprob*np.log2(infoprob))
      entropy+=prob*infoGain
    return entropy

  def informationGain(self,df):
    total=len(df.index)
    column=df.iloc[:,-1].name
    valueCounts=self.valueCountsDict(df,column)
    infoGain=0
    for category in valueCounts.keys():
      prob=valueCounts[category]/total
      if prob!=0:
        infoGain-=(prob*np.log2(prob))
    return infoGain

  def getSplittingFeature(self,df):
    featureEntropyDict={}
    featureGainDict={}
    for feature in df.iloc[:,0:-1].columns:
      featureEntropyDict[feature]=self.entropy(df,feature)
      featureGainDict[feature]=self.informationGain(df)-self.entropy(df,feature)
    maxGainValue=max(featureGainDict.values())
    for splittingFeature,gainValue in featureGainDict.items():
      if gainValue==maxGainValue:
        return splittingFeature,gainValue,featureEntropyDict[splittingFeature]

  def buildTree(self, node=None):
          if node is None:
              df = self.X
              informationGain = self.informationGain(df)
              splittingFeature, gainValue, entropyValue = self.getSplittingFeature(df)
              root = Node(value=splittingFeature, informationGain=informationGain,
                          entropy=entropyValue, gain=gainValue)
              self.tree.node(root.value)
              root.setDefaultLinks(self.X)
              self.root = root
              self.queue.append(root)
          else:
              for link, value in node.linksDict.items():
                  if value is None:
                      df = self.X
                      dfConstraints = node.dfConstraints.copy()
                      dfConstraints[node.value] = link
                      for key, value in dfConstraints.items():
                          df = df[df[key] == value]
                      df = df.drop(dfConstraints.keys(), axis=1)
                      if not df.empty:
                          informationGain = self.informationGain(df)
                          if informationGain == 0:
                              nodeValue = df.iloc[0, -1]
                              newnode = Node(value=nodeValue)
                              node.linksDict[link] = newnode
                              self.tree.node(link, label=newnode.value)
                              self.tree.edge(node.value, link, label=link)
                          else:
                              splittingFeature, gainValue, entropyValue = self.getSplittingFeature(df)
                              newnode = Node(value=splittingFeature, informationGain=informationGain,
                                            entropy=entropyValue, gain=gainValue, dfConstraints=dfConstraints)
                              newnode.setDefaultLinks(self.X)
                              newnode.dfConstraints = dfConstraints
                              node.linksDict[link] = newnode
                              self.tree.node(newnode.value)
                              self.tree.edge(node.value, newnode.value, label=link)
                              self.queue.append(newnode)

  def fit(self,X,y):
    self.X=X
    self.y=y
    self.df=X
    self.df[y.name]=y
    self.buildTree()
    while self.queue != []:
      node=self.queue.pop(0)
      # print(node.value,":")
      self.buildTree(node)
    self.tree.render('DecisionTree_predict.dot', format='png')

  def predict(self, X):
      predictions = []
      for _, row in X.iterrows():
          current_node = self.root
          while current_node.linksDict:
              feature_value = row[current_node.value]
              current_node = current_node.linksDict[feature_value]
          predictions.append(current_node.value)
      return predictions
tree=graphviz.Digraph()
t=DecisionTree(tree=tree)
t.fit(X,y)

In [ ]:
class Node:
  def __init__(self,value,informationGain=None,entropy=None,gain=None,dfConstraints={},class_distribution={}):
    self.value=value
    self.informationGain=informationGain
    self.entropy=entropy
    self.gain=gain
    self.linksDict={}
    self.isLeaf=False
    self.dfConstraints=dfConstraints
    self.class_distribution = class_distribution
  def setDefaultLinks(self,df):
    for category in df[self.value].unique():
      self.linksDict[category]=None

class DecisionTree:
  def __init__(self,tree,alpha=0.2):
    self.root=None
    self.tree=tree
    self.queue=[]
    self.alpha = alpha

  def valueCountsDict(self,df,column):
    valueCounts=np.unique(df[column],return_counts=True)
    valueCountsDict = dict(zip(valueCounts[0],valueCounts[1]))
    return valueCountsDict

  def entropy(self,df,column):
    entropy=0
    tempdf=pd.DataFrame(columns=self.valueCountsDict(df,df.iloc[:,-1].name).keys(),index=self.valueCountsDict(df,column).keys())
    for x in self.valueCountsDict(df,column).keys():
      for y in self.valueCountsDict(df,df.iloc[:,-1].name).keys():
        tempdf.loc[x,y]=len(df[(df[column]==x) & (df[df.iloc[:,-1].name]==y)].index)
    valueCounts=self.valueCountsDict(df,column)
    targetValueCounts=self.valueCountsDict(df,df.iloc[:,-1].name)
    total=len(df.index)
    for category in valueCounts.keys():
      prob=valueCounts[category]/total
      infoGain=0
      for targetCategory in targetValueCounts.keys():
        infoprob=tempdf.loc[category,targetCategory]/tempdf.loc[category].sum()
        if infoprob!=0:
          infoGain-=(infoprob*np.log2(infoprob))
      entropy+=prob*infoGain
    return entropy

  def informationGain(self,df):
    total=len(df.index)
    column=df.iloc[:,-1].name
    valueCounts=self.valueCountsDict(df,column)
    infoGain=0
    for category in valueCounts.keys():
      prob=valueCounts[category]/total
      if prob!=0:
        infoGain-=(prob*np.log2(prob))
    return infoGain

  def getSplittingFeature(self,df):
    featureEntropyDict={}
    featureGainDict={}
    for feature in df.iloc[:,0:-1].columns:
      featureEntropyDict[feature]=self.entropy(df,feature)
      featureGainDict[feature]=self.informationGain(df)-self.entropy(df,feature)
    maxGainValue=max(featureGainDict.values())
    for splittingFeature,gainValue in featureGainDict.items():
      if gainValue==maxGainValue:
        return splittingFeature,gainValue,featureEntropyDict[splittingFeature]

  def buildTree(self, node=None):
        if node is None:
            df = self.X
            informationGain = self.informationGain(df)
            splittingFeature, gainValue, entropyValue = self.getSplittingFeature(df)
            root = Node(value=splittingFeature, informationGain=informationGain,
                        entropy=entropyValue, gain=gainValue,
                        class_distribution=self.valueCountsDict(df, df.iloc[:, -1].name))  # Pass class distribution
            self.tree.node(root.value)
            root.setDefaultLinks(self.X)
            self.root = root
            self.queue.append(root)
        else:
            for link, value in node.linksDict.items():
                if value is None:
                    df = self.X
                    dfConstraints = node.dfConstraints.copy()
                    dfConstraints[node.value] = link
                    for key, value in dfConstraints.items():
                        df = df[df[key] == value]
                    df = df.drop(dfConstraints.keys(), axis=1)
                    if not df.empty:
                        informationGain = self.informationGain(df)
                        if informationGain == 0:
                            nodeValue = df.iloc[0, -1]
                            newnode = Node(value=nodeValue)
                            node.linksDict[link] = newnode
                            self.tree.node(link, label=newnode.value)
                            self.tree.edge(node.value, link, label=link)
                        else:
                            splittingFeature, gainValue, entropyValue = self.getSplittingFeature(df)
                            newnode = Node(value=splittingFeature, informationGain=informationGain,
                                           entropy=entropyValue, gain=gainValue,
                                           dfConstraints=dfConstraints,
                                           class_distribution=self.valueCountsDict(df, df.iloc[:, -1].name))  # Pass class distribution
                            newnode.setDefaultLinks(self.X)
                            newnode.dfConstraints = dfConstraints
                            node.linksDict[link] = newnode
                            self.tree.node(newnode.value)
                            self.tree.edge(node.value, newnode.value, label=link)
                            self.queue.append(newnode)
  # def fit(self,X,y):
  #   self.X=X
  #   self.y=y
  #   self.df=X
  #   self.df[y.name]=y
  #   self.buildTree()
  #   while self.queue != []:
  #     node=self.queue.pop(0)
  #     # print(node.value,":")
  #     self.buildTree(node)
  #   self.tree.render('DecisionTree.dot', format='png')

  def predict(self, X):
      predictions = []
      for _, row in X.iterrows():
          current_node = self.root
          while current_node.linksDict:
              feature_value = row[current_node.value]
              current_node = current_node.linksDict[feature_value]
          predictions.append(current_node.value)
      return predictions

  def fit(self, X, y,alpha=0.2):
        self.alpha = alpha
        self.X = X
        self.y = y
        self.df = X
        self.df[y.name] = y
        self.buildTree()
        while self.queue:
            node = self.queue.pop(0)
            self.buildTree(node)
        # Perform cost complexity pruning
        self.cost_complexity_pruning(self.root)
        self.tree.render('Pruned_DecisionTree', format='png')

  def cost_complexity_pruning(self, node):
    if node is None:
        return
    # Recursively calculate cost complexity for each subtree
    for link, child_node in node.linksDict.items():
        self.cost_complexity_pruning(child_node)
    # Calculate impurity for current subtree
    impurity = self.calculate_impurity(node)
    # Calculate number of leaves in current subtree
    num_leaves = self.count_leaves(node)
    # Calculate cost complexity parameter
    cost_complexity = impurity + self.alpha * (num_leaves - 1)  # Subtract 1 to exclude the root node
    node.cost_complexity = cost_complexity

  def calculate_impurity(self, node):
    if node is None:
        return 0
    total_samples = sum(node.class_distribution.values())
    impurity = 1.0
    for class_count in node.class_distribution.values():
        proportion = class_count / total_samples
        impurity -= proportion ** 2
    return impurity

  def count_leaves(self, node):
        # Recursively count the number of leaves in the subtree
        if node is None:
            return 0
        if node.isLeaf:
            return 1
        num_leaves = 0
        for link, child_node in node.linksDict.items():
            num_leaves += self.count_leaves(child_node)
        return num_leaves
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.3, random_state=42)

tree = DecisionTree(tree=graphviz.Digraph())

tree.fit(X_train, y_train)

# Make predictions on the testing set
test_predictions = tree.predict(X_test)

y_val_pred = tree.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Validation set accuracy:", val_accuracy)

# Calculate the accuracy on the testing set
test_accuracy = (test_predictions == y_test).mean()
print("Testing set accuracy:", test_accuracy)

In [ ]:
import numpy as np
import pandas as pd
import pydot

class Node:
    def __init__(self, attribute=None, threshold=None, left=None, right=None, value=None,mse=None):
        self.attribute = attribute  # Splitting attribute
        self.threshold = threshold  # Threshold for splitting
        self.left = left  # Left child
        self.right = right  # Right child
        self.value = value  # Predicted value if it's a leaf node
        self.mse = mse #squared error at each node
def sum_squared_error(targets):
    if len(targets) == 0:
        return 0
    mean = np.mean(targets)
    return np.sum((targets - mean) ** 2)

def construct_regression_tree(df, target_attribute, ats):
    if len(ats) == 0:
        # If no attributes left, return leaf node with mean target value
        return Node(value=np.mean(df[target_attribute]))

    best_score = float('inf')#representing integer as infinity in python
    best_attribute = None
    best_threshold = None

    for x in ats:
        if x is None:
            continue  # Skip None attributes

        # Calculate mean squared error for each attribute and its possible thresholds
        values = df[x].values
        thresholds = np.unique(values)
        for y in thresholds:
            left_examples = df[df[x] <= y]
            right_examples = df[df[x] > y]

            left_targets = left_examples[target_attribute].values
            right_targets = right_examples[target_attribute].values

            score = sum_squared_error(left_targets) + sum_squared_error(right_targets)
            if score < best_score:
                best_score = score
                best_attribute = x
                best_threshold = y

    if best_attribute is None:
        # If best_attribute is still None, return leaf node with mean target value
        return Node(value=np.mean(df[target_attribute]))

    left_examples = df[df[best_attribute] <= best_threshold]
    right_examples = df[df[best_attribute] > best_threshold]

    remaining_attributes = [x for x in ats if x != best_attribute]

    left_subtree = construct_regression_tree(left_examples, target_attribute, remaining_attributes)
    right_subtree = construct_regression_tree(right_examples, target_attribute, remaining_attributes)

    return Node(attribute=best_attribute, threshold=best_threshold, left=left_subtree, right=right_subtree,mse=best_score)

def visualize_regression_tree(node, graph=None):
    if graph is None:
        graph = pydot.Dot(graph_type='digraph')

    if node.attribute is not None:#if it is not a leaf node
        if node.value is not None:# If it has a value (e.g., for decision nodes)
            label = f"{node.attribute}\n<= {node.threshold}\nMSE: {node.value:.2f}"
        else:
            label = f"{node.attribute}\n<= {node.threshold}\nMSE: {node.mse:.2f}"
    else:
        label = f"Predicted value: {node.value:.2f}"

    node_name = str(id(node))# Unique node name using its memory address
    graph.add_node(pydot.Node(node_name, label=label))# Add node to graph

    if node.left is not None:
        left_child_name = str(id(node.left))
        #print(left_child_name)
        graph.add_node(pydot.Node(left_child_name, label="")) # Add placeholder for label update later
        graph.add_edge(pydot.Edge(node_name, left_child_name, label="True"))# Connect with edge
        visualize_regression_tree(node.left, graph)# Recursively visualize subtree

    if node.right is not None:
        right_child_name = str(id(node.right))
        graph.add_node(pydot.Node(right_child_name, label=""))
        graph.add_edge(pydot.Edge(node_name, right_child_name, label="False"))
        visualize_regression_tree(node.right, graph)

    return graph

    # Load dataset from CSV
df = pd.read_csv("Reg_tree.csv")

    # Specify target attribute and attributes
target_attribute = df.columns[-1]
#"Play"
ats = df.columns[0:4]
 #["Outlook", "Temperature","Humidity","Wind"]

    # Construct the regression tree
tree = construct_regression_tree(df, target_attribute, ats)

    # Visualize the regression tree
graph = visualize_regression_tree(tree)
graph.write_png('regression.png')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

class Layer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 2 * np.random.random((n_inputs, n_neurons)) - 1
        self.biases = 2 * np.random.random((1, n_neurons)) - 1

    def calculate_output(self, input):
        self.input = np.array(input)
        weighted_sum = np.dot(self.input, self.weights) + self.biases
        self.output = self.activation_function(weighted_sum)
        return self.output

    def calculate_delta(self, error, output):
        return error * self.derivative_activation_function(output)

    def update_weights(self, input, delta, learning_rate):
        if delta.shape[0] != input.shape[0]:
            input = input.reshape(1, -1)
        self.weights += learning_rate * np.dot(delta.T, input).T
        self.biases += learning_rate * np.sum(delta)

    def activation_function(self, x):
        return sigmoid(x)

    def derivative_activation_function(self, x):
        return sigmoid_derivative(x)

class NeuralNetwork:
    def __init__(self, layers):
        layer = []
        for i in range(0, len(layers) - 1):
            layer.append(Layer(layers[i], layers[i + 1]))
        self.layers = layer
        self.itr = []
        self.err = []

    def predict(self, input):
        output = input
        for layer in self.layers:
            output = layer.calculate_output(output)
        return np.round(output,6)

    def predict_(self, input):
        output = input
        for layer in self.layers:
            output = layer.calculate_output(output)
        return np.round(output)

    def calculate_mse_error(self, expected_output, output):
        return np.mean((expected_output - output) ** 2)

    def calculate_binary_cross_entropy_error(self, expected_output, output):
        return -np.mean(expected_output * np.log(output) + (1 - expected_output) * np.log(1 - output))

    def calculate_categorical_cross_entropy_error(self, expected_output, output):
        return -np.mean(expected_output * np.log(output))

    def train(self, input, expected_output, learning_rate, error_function):
        output = self.predict(input)
        error = expected_output - output
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]
            input = layer.input
            output = layer.output
            if i == len(self.layers) - 1:
                layer.delta = layer.calculate_delta(error, output)
            else:
                next_layer = self.layers[i + 1]
                layer.delta = layer.calculate_delta(np.dot(next_layer.delta, next_layer.weights.T), output)
            self.layers[i] = layer

        for i in range(0, len(self.layers)):
            layer = self.layers[i]
            layer.update_weights(layer.input, layer.delta, learning_rate)
            self.layers[i] = layer


        if error_function == 'mse':
            error = self.calculate_mse_error(expected_output, self.predict(input))
        elif error_function == 'binary_cross_entropy':
            error = self.calculate_binary_cross_entropy_error(expected_output, self.predict(input))
        elif error_function == 'categorical_cross_entropy':
            error = self.calculate_categorical_cross_entropy_error(expected_output, self.predict(input))

        return error

    def plot(self):
        plt.plot(self.itr, self.err)
        plt.title("Iterations vs Error")
        plt.xlabel("Iterations")
        plt.ylabel("Error")

    def plot1(self, lr, itr):
        plt.plot(lr, itr)
        plt.title("Learning Rate vs No of Iterations")
        plt.xlabel("Learning Rate")
        plt.ylabel("No of Iterations")

    def fit(self, e, lr, error_function):
        for i in range(e):
            for j in range(len(X)):
                self.train(X[j], y[j], lr, error_function)
            if i % 10000 == 0:
                self.itr.append(i)
                error = abs(float(y[0] - float(nn.predict(X[0]))))
                self.err.append(error)
                print(f"Iteration: {i}, Error: {error}")
        final_error = abs(float(y[0] - float(nn.predict(X[0]))))
        print(f"Final Error: {final_error}")

X = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
y = np.array([[0], [1], [1], [0], [1], [0], [0], [1]])
nn = NeuralNetwork([3, 3, 1])

nn.fit(100000, 0.7, 'binary_cross_entropy')
print(nn.predict_(np.array([0, 0, 0])))
print(nn.predict_(np.array([0, 0, 1])))
print(nn.predict_(np.array([0, 1, 0])))
print(nn.predict_(np.array([0, 1, 1])))
print(nn.predict_(np.array([1, 0, 0])))
print(nn.predict_(np.array([1, 0, 1])))
print(nn.predict_(np.array([1, 1, 0])))
print(nn.predict_(np.array([1, 1, 1])))

nn.plot()


In [ ]:
max_number = max(max(sublist) for sublist in X)
result_list = [[element / max_number for element in sublist] for sublist in X]
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(result_list,y)
import numpy as np
class Layer:
  def __init__(self,nNeurons,weights,bias,activation_function,activation_derivative):
    self.nNeurons = nNeurons
    self.weights = weights
    self.bias = bias
    self.activation_function = activation_function
    self.activation_derivative = activation_derivative
    self.delta = None
  def calculate_output(self,data,input_Layer=False):
    if(input_Layer):
      self.output = data
    else:
      self.weighted_sum = np.dot(data,self.weights)+self.bias
      self.output = np.array(self.activation_function(self.weighted_sum))
    return self.output


class Network:
  def __init__(self,nLayers):
    self.nLayers=nLayers
    self.Layers=[]
    self.pred_output=[]


  def sigmoid(self,x):
    return 1/(1+np.exp(x))
  def sigmoid_derivative(self,x):
    return self.sigmoid(x)*(1-self.sigmoid(x))
  def input_act(self,x):
    return x
  def relu(self,x):
    return np.maximum(0,x)
  def relu_derivative(self,x):
    return np.where(x < 0, 0, 1)
  def softmax(self,x):
    exp_values=np.exp(x)
    expSum=np.sum(exp_values)
    return exp_values/expSum
  def softmax_derivative(self,x):
    return self.softmax(x) * (1 - self.softmax(x))



  def Create_Network(self,nInputs):
    for i in range(self.nLayers-1):
      if i==0:
        weights=np.ones(nInputs)
        bias=np.ones((nInputs))
        self.Layers.append(Layer(nInputs,weights,bias,self.input_act, self.input_act))
      else:
        nNeurons=int(input(f"enter number of neurons of Layer {i+1} "))
        weights=0.1*np.random.randn(nInputs,nNeurons)
        bias=0.1*np.random.randn((nNeurons))
        self.Layers.append(Layer(nNeurons,weights,bias,self.relu, self.relu_derivative))
        nInputs=nNeurons
    nNeurons=int(input(f"enter number of neurons in output layer "))
    weights=0.1*np.random.randn(nInputs,nNeurons)
    bias=np.zeros((nNeurons))
    self.Layers.append(Layer(nNeurons,weights,bias,self.softmax, self.softmax_derivative))


  def forwardPass(self,Input_data):
    for i in range(self.nLayers):
      if(i==0):
        Input_data=self.Layers[i].calculate_output(Input_data,True)
      else:
        Input_data=self.Layers[i].calculate_output(Input_data)
    return Input_data


  def calculate_deltas(self, targets):
    for i in range(self.nLayers - 1, 0, -1):
      if i == len(self.Layers)-1:
        self.Layers[i].delta = (self.Layers[i].output - targets)
      else:
        self.Layers[i].delta = np.dot(self.Layers[i+1].delta,self.Layers[i+1].weights.T) * self.Layers[i].activation_derivative(self.Layers[i].weighted_sum)

  def Update_Weights(self,lr):
    for i in range(self.nLayers - 1, 0, -1):
      self.Layers[i].bias -= np.dot(lr, self.Layers[i].delta)
      self.Layers[i].weights -= self.Layers[i].delta[np.newaxis,:] * (np.dot(lr,self.Layers[i-1].output)[:, np.newaxis]  * self.Layers[i].weights)

  def calculate_error(self, targets, outputs):
      epsilon = 1e-15
      outputs = np.clip(outputs, epsilon, 1 - epsilon)
      loss = - np.sum(targets * np.log(outputs))
      return loss

  def backwardPass(self,targets, lr):
    self.calculate_deltas(targets)
    self.Update_Weights(lr)

  def One_hot(self,x):
    ans = []
    for i in range(4):
      if(i==x):
        ans.append(1)
      else:
        ans.append(0)
    return np.array(ans)

  def fit(self, input_datas, target_labels, epochs, learning_rate):
      for epoch in range(epochs):
        total_error = 0
        for input_data,target in zip(input_datas, target_labels):
          outputs = self.forwardPass(input_data)
          targets = self.One_hot(target)
          error = self.calculate_error(targets, outputs)
          total_error+=error
          self.backwardPass(targets, learning_rate)
        print(f"Epoch {epoch + 1} / {epochs}, Error: {total_error/len(input_datas)}")

nLayers=int(input("enter number of layers"))

obj=Network(nLayers)  #object for Network class
obj.Create_Network(4096)  #creates a network by creating layers and appending to Layers list in Network class (this takes input length as argument)


In [ ]:
import pandas as pd
import numpy as np
import pydot
from IPython.display import Image, display
class Neuron:
    def __init__(self, weights, activation_function):
        self.weights = weights
        self.activation_function = activation_function
        self.output = None

    def compute_output(self, inputs):
        weighted_sum = np.dot(inputs, self.weights)
        self.output = self.activation_function(weighted_sum)
        return self.output

class Layer:
    def __init__(self, num_neurons, num_inputs_per_neuron, activation_function):
        self.neurons = [Neuron(np.random.rand(num_inputs_per_neuron), activation_function) for _ in range(num_neurons)]

    def compute_outputs(self, inputs):
        return [neuron.compute_output(inputs) for neuron in self.neurons]

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward_pass(self, input_data):
        outputs = input_data
        for layer in self.layers:
            outputs = layer.compute_outputs(outputs)
        return outputs

    def visualize(self):
        graph = pydot.Dot(graph_type='digraph')
        for i, layer in enumerate(self.layers):
            for j, neuron in enumerate(layer.neurons):
                graph.add_node(pydot.Node(f'L{i}N{j}', label=f'Layer {i}\nNeuron {j}\n Value x({j}):{neuron.output:.2f}'))

                if i > 0:
                    for k, prev_neuron in enumerate(self.layers[i - 1].neurons):
                        graph.add_edge(pydot.Edge(f'L{i-1}N{k}', f'L{i}N{j}', label=f'W={neuron.weights[k]:.2f}'))
        print(graph)
        graph.write("neural_network.dot")
        graph.write_png('neural_network.png')
        display(Image('neural_network.png'))
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Create layers with appropriate number of neurons and inputs per neuron
input_layer = Layer(num_neurons=3, num_inputs_per_neuron=3, activation_function=sigmoid)
hidden_layer = Layer(num_neurons=4, num_inputs_per_neuron=3, activation_function=sigmoid)
hidden_layer1 = Layer(num_neurons=4, num_inputs_per_neuron=4, activation_function=sigmoid)
output_layer = Layer(num_neurons=2, num_inputs_per_neuron=4, activation_function=sigmoid)

# Create a neural network and add layers
neural_network = NeuralNetwork()
neural_network.add_layer(input_layer)
neural_network.add_layer(hidden_layer)
neural_network.add_layer(hidden_layer1)
neural_network.add_layer(output_layer)

# Example input data
input_data = np.array([0.5, 0.3, 0.1])

output = neural_network.forward_pass(input_data)
print("Final Output:", output)

neural_network.visualize()

In [ ]:
@dataclass
class Neuron:
  type:str=None
  weights:list=field(default_factory=list)
  inputs:list=field(default_factory=list)
  outputs:list=field(default_factory=list)
  activationfunc=sigmoid
  DEbyDX=None
  DEbyDY=None
  def computeipoutput(self):
        if self.type=="input":
          self.weights=None
          self.inputs=None
  def computeoutput(self):
    pass
@dataclass
class Layer:
  type:str=None
  listofneurons:list=field(default_factory=list)
  weights:list=field(default_factory=list)
  #inputs:list=field(default_factory=list)
  outputs:list=field(default_factory=list)
  activationfunc:sigmoid=sigmoid
  def computeoutput(self,input):
    if self.type!="input":
      netsum=np.dot(input,self.weights)
      print(netsum)
      self.outputs=self.activationfunc(netsum)
      return self.outputs
    else:
        self.outputs=input
@dataclass
class Network:
  listoflayers:list=field(default_factory=list)
  def forwardpass(self,input):
    for i in range(len(self.listoflayers)-1):
      weights=np.ones((len(self.listoflayers[i].listofneurons),len(self.listoflayers[i+1].listofneurons)))
      self.listoflayers[i+1].weights=weights

    self.listoflayers[0].computeoutput(input)
    self.listoflayers[1].inputs=input
    activation=input
    for i in range(1,len(self.listoflayers)):
      print(activation)
      activation=self.listoflayers[i].computeoutput(activation)
    print(activation)
MLP=Network()
ipneurons=int(input("enter no of i/p neurons"))
iplayer=Layer(type="input")
for i in range(ipneurons):
    neuron=Neuron(type="input")
    iplayer.listofneurons.append(Neuron)
MLP.listoflayers.append(iplayer)

hidlay=int(input("enter no of hidden layers"))
for i in range(hidlay):
   hiddenlayer=Layer()
   hidneu=int(input("enter no of hidden neurons"))
   for i in range(hidneu):
     neuron=Neuron()
     hiddenlayer.listofneurons.append(Neuron)
   MLP.listoflayers.append(hiddenlayer)



In [ ]:
MLP=Network()
ipneurons=int(input("enter no of i/p neurons"))
iplayer=Layer(type="input")
for i in range(ipneurons):
    neuron=Neuron(type="input")
    iplayer.listofneurons.append(Neuron)
MLP.listoflayers.append(iplayer)

hidlay=int(input("enter no of hidden layers"))
for i in range(hidlay):
   hiddenlayer=Layer()
   hidneu=int(input("enter no of hidden neurons"))
   for i in range(hidneu):
     neuron=Neuron()
     hiddenlayer.listofneurons.append(Neuron)
   MLP.listoflayers.append(hiddenlayer)
@dataclass
class Layer:
    noofneurons:int=None
    neurons:list=field(default_factory=list)
    weights:np.array=None
    typ:str=None
    input:np.array=None
    output:np.array=None
    def initial(self,typ=None,prevlayer=None):
        if prevlayer==None:
                for i in range(self.noofneurons):
                    n=Neuron(typ=self.typ)
                    self.neurons.append(n)

        else:
            for i in range(self.noofneurons):
                    n=Neuron(typ=typ)
                    self.neurons.append(n)

            #for n in prevlayer.neurons:
                #n.weights=np.ones((len(self.neurons),))
                #n.weightsin=np.ones((len(self.neurons),))
            self.weights=np.ones((len(prevlayer.neurons),len(self.neurons)))
    def computeoutput(self,inputs):
        return [neuron.computeoutput(inputs) for neuron in self.neurons]
@dataclass
class NeuralNetwork:
    layers:list=field(default_factory=list)
    def add(self,layer,typ):
        if typ=='input':
            self.layers.append(layer)
            layer.initial(typ='input')
        if typ=='hidden' or  typ=='output':
            self.layers.append(layer)
            layer.initial(typ,self.layers[-2])
    def forwardpass(self,inputdata):
        self.layers[0].input= inputdata
        self.layers[0].output= inputdata
        output=self.layers[0].output
        for i in range(1,len(self.layers)):
            weightedsum=np.dot(self.layers[i].weights.T,output)
            self.layers[i].input=weightedsum
            self.layers[i].output=sigmoid(weightedsum)
            output=self.layers[i].output
        return output
n=NeuralNetwork()
n.add(Layer(1),typ='input')
n.add(Layer(2),typ='hidden')
n.add(Layer(2),typ='hidden')


